# **BBM409 ASSIGNMENT 3**

        Eylül TUNCEL - 21727801
        Emre KÖSEN   - 21727498

In this assignment, we will try to determine whether a mail is ham or spam with Naive Bayes classifier. Naive Bayes is a simple classification algorith that makes an assumption about the conditional independence of features. In our dataset, there are two columns (text, spam).
* ”text”: the text of the article, could be incomplete
* ”spam”: a label that marks the article as potentially spam or ham
    - 1: spam
    - 0: ham

In [ ]:
import math
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import ENGLISH_STOP_WORDS

In [ ]:
def split_data(x):
    # start and end points of each fold
    size = int(x.shape[0] / 5)
    # size = 100

    # 1/5 part of the data set as test data
    x_test = x[0:size]
    print(x_test.shape)

    # 4/5 part of the data set as test data
    x_train = x[size:]
    print(x_train.shape)
    return x_test, x_train


In [ ]:
def vectorizer(x, n_gram):
    # text_col refers to the first column which has mail texts
    text_col = []
    for i in range(x.shape[0]):
        text_col.append(x[i, 0])

    # initialize count vectorizer
    if n_gram == 1:
        count_vectorizer = CountVectorizer()
    elif n_gram == 2:
        count_vectorizer = CountVectorizer(analyzer='word', ngram_range=(2, 2))
    # matrix of token counts
    matrix = count_vectorizer.fit_transform(text_col)

    # vocabulary is the array of unique words which appears in all train texts
    vocabulary = count_vectorizer.get_feature_names_out()
    matrix = matrix.toarray()

    # unique words dictionary is a dict which has all unique words as key and an array [spam count, ham count] as value
    # one example key value pair is { "word" : [ 12, 30 ] ,  , }
    unique_words_dict = dict.fromkeys(vocabulary)
    for key in unique_words_dict:
        # initialize spam and ham count as [0,0] at the beginning
        unique_words_dict[key] = [0, 0]

    count_spam_mails = 0
    count_ham_mails = 0
    total_spam_words = 0
    total_ham_words = 0

    # for all train samples
    for i in range(len(x)):

        # if the sample is HAM
        if x[i][1] == 0:
            count_ham_mails += 1
            for j in range(len(matrix[i])):
                if matrix[i][j] != 0:
                    # increase count by one in unique words dictionary
                    w = vocabulary[j]
                    unique_words_dict[w] = [unique_words_dict.get(w)[0], unique_words_dict.get(w)[1] + 1]
                    total_ham_words += 1

        # if the sample is SPAM
        else:
            count_spam_mails += 1
            for j in range(len(matrix[i])):
                if matrix[i][j] != 0:
                    w = vocabulary[j]
                    unique_words_dict[w] = [unique_words_dict.get(w)[0] + 1, unique_words_dict.get(w)[1]]
                    total_spam_words += 1

    # total mail count of spam and ham mails
    count_mails = [count_spam_mails, count_ham_mails]

    # total word count appeared in spam and ham mails
    total_words_dist = [total_spam_words, total_ham_words]

    return unique_words_dict, count_mails, total_words_dist


In [ ]:
def max_prob_words_by_naive_bayes(unique_words_dict, total_words_dist):
    max_spam_count = 0
    max_spam_word1 = ""
    max_spam_word2 = ""
    max_spam_word3 = ""
    max_ham_count = 0
    max_ham_word1 = ""
    max_ham_word2 = ""
    max_ham_word3 = ""

    for x in unique_words_dict.keys():
        arr = unique_words_dict.get(x)

        # spam
        if arr[0] > max_spam_count and ((arr[1] / total_words_dist[1]) / (arr[0] / total_words_dist[0])) < 0.5:
            max_spam_count = arr[0]
            max_spam_word3 = max_spam_word2
            max_spam_word2 = max_spam_word1
            max_spam_word1 = x

        # ham
        if arr[1] > max_ham_count and ((arr[0] / total_words_dist[0]) / (arr[1] / total_words_dist[1])) < 0.3:
            max_ham_count = arr[1]
            max_ham_word3 = max_ham_word2
            max_ham_word2 = max_ham_word1
            max_ham_word1 = x

    print("max ham1 : ", max_ham_word1)
    print("max ham2 : ", max_ham_word2)
    print("max ham3 : ", max_ham_word3)
    print()
    print("max spam1 : ", max_spam_word1)
    print("max spam2 : ", max_spam_word2)
    print("max spam3 : ", max_spam_word3)
    print()
    return

In [ ]:
def calculate_probability(x_test, unique_words_dict, count_mails, total_words_dist, n_gram):
    # take all the texts of mails in test data
    text_col = []
    for i in range(x_test.shape[0]):
        text_col.append(x_test[i, 0])

    # initialize count vectorizer
    if n_gram == 1:
        count_vectorizer = CountVectorizer()
    elif n_gram == 2:
        count_vectorizer = CountVectorizer(analyzer='word', ngram_range=(2, 2))
    # matrix of token counts in test data
    matrix = count_vectorizer.fit_transform(text_col)
    vocabulary = count_vectorizer.get_feature_names_out()
    matrix = matrix.toarray()

    # probability of being spam
    prob_spam = total_words_dist[0] / (total_words_dist[0] + total_words_dist[1])
    # probability of being ham
    prob_ham = total_words_dist[1] / (total_words_dist[0] + total_words_dist[1])

    predictions = []
    results = {}

    # Iterate over all test samples
    for i in range(x_test.shape[0]):

        # take log of the probability of being spam mail and being ham mail
        probability_spam = math.log2(prob_spam)
        probability_ham = math.log2(prob_ham)

        # Iterate over all words of one test sample
        for j in range(len(matrix[i])):

            # if matrix[i][j] is a number different than zero that means this word appears in that train sample
            if matrix[i][j] != 0:

                # Spam and ham count starts from 1 because of laplace smoothing
                spam_count = 1
                ham_count = 1

                # We add number of unique words for laplace smoothing
                spam_denominator = total_words_dist[0] + len(unique_words_dict)
                ham_denominator = total_words_dist[1] + len(unique_words_dict)

                word = unique_words_dict.get(vocabulary[j])

                # Check whether word is in training samples or not
                if word is not None:
                    spam_count += word[0]
                    ham_count += word[1]

                # take log of the probabilities and sum them up
                probability_spam += math.log2(spam_count / spam_denominator)
                probability_ham += math.log2(ham_count / ham_denominator)

        # print(probability_spam, probability_ham)

        # by the naive bayes algorithm , take maximum probability as prediction class
        if probability_spam > probability_ham:
            predictions.append(1)
        else:
            predictions.append(0)

        # result array has two dimensional arrays in it for each test sample
        # test sample x = [ actual class, predicted class ]
        results[i] = [x_test[i][1], predictions[i]]

    return results


In [ ]:
def tf_idf(x):
    text_col_spam = []
    text_col_ham = []
    for i in range(x.shape[0]):
        if x[i][1] == 1:
            text_col_spam.append(x[i, 0])
        elif x[i][1] == 0:
            text_col_ham.append(x[i, 0])

    count_vectorizer_spam = CountVectorizer()
    matrix = count_vectorizer_spam.fit_transform(text_col_spam)
    vocabulary_spam = count_vectorizer_spam.get_feature_names_out()
    pipe = Pipeline([('count', CountVectorizer(vocabulary=vocabulary_spam)), ('tfidf', TfidfTransformer())]).fit(
        text_col_spam)

    spam_tf_idf_arr = pipe['tfidf'].idf_
    spam_tf_idf_dict = {}
    for i in range(len(spam_tf_idf_arr)):
        spam_tf_idf_dict[vocabulary_spam[i]] = spam_tf_idf_arr[i]

    spam_tf_idf_arr = sorted(spam_tf_idf_arr)
    spam_words = []
    for i in range(100):
        val = spam_tf_idf_arr[i]
        for el in spam_tf_idf_dict.keys():
            if spam_tf_idf_dict.get(el) == val:
                spam_words.append(el)
                break

    # Ham part
    count_vectorizer_ham = CountVectorizer()
    matrix = count_vectorizer_ham.fit_transform(text_col_ham)
    vocabulary_ham = count_vectorizer_ham.get_feature_names_out()
    pipe = Pipeline([('count', CountVectorizer(vocabulary=vocabulary_ham)), ('tfidf', TfidfTransformer())]).fit(
        text_col_ham)

    ham_tf_idf_arr = pipe['tfidf'].idf_
    ham_tf_idf_dict = {}
    for i in range(len(ham_tf_idf_arr)):
        ham_tf_idf_dict[vocabulary_ham[i]] = ham_tf_idf_arr[i]

    ham_tf_idf_arr = sorted(ham_tf_idf_arr)
    ham_words = []
    for i in range(100):
        val = ham_tf_idf_arr[i]
        for el in ham_tf_idf_dict.keys():
            if ham_tf_idf_dict.get(el) == val:
                ham_words.append(el)
                break

    spam_words = list(dict.fromkeys(spam_words))
    ham_words = list(dict.fromkeys(ham_words))
    a, b = [spam_words, ham_words]
    s = [x for x in b if x in a]
    for i in range(len(s)):
        word = s[i]
        if (spam_tf_idf_dict.get(word) - ham_tf_idf_dict.get(word)) < 1.5:
            if spam_words.count(word) > 0:
                spam_words.remove(word)
            if ham_words.count(word) > 0:
                ham_words.remove(word)

    print("spam words", spam_words[:10])
    print("ham words", ham_words[:10])
    return

In [ ]:
# while calculating performance metrics
# True positive = th -> truly predicted ham mail
# False positive = fh -> falsely predicted ham mail
# True negative = ts -> truly predicted spam mail
# False negative = fs -> falsely predicted spam mail
def calculate_performance(results):
    th = 0
    ts = 0
    fh = 0
    fs = 0
    for key, value in results.items():
        # if the mail is ham and predicted as ham
        if value[0] == value[1] and value[1] == 0:
            th += 1
        # if the mail is spam and predicted as spam
        elif value[0] == value[1] and value[1] == 1:
            ts += 1
        # if the mail is spam but predicted as ham
        if value[0] != value[1] and value[1] == 0:
            fh += 1
        # if the mail is ham but predicted as spam
        elif value[0] != value[1] and value[1] == 1:
            fs += 1

    accuracy = (th + ts) / (th + ts + fh + fs)
    precision = th / (th + fh)
    recall = th / (th + fs)
    f1_score = (2 * recall * precision) / (recall + precision)
    return accuracy, precision, recall, f1_score


In [15]:
def main(total_words_dis=None):
    # reading data's in the csv file to the numpy array
    df = pd.read_csv('./emails.csv')
    x = np.array(df.iloc[:, :])

    # shuffle the data
    np.random.seed(101)
    np.random.shuffle(x)
    np.random.seed(102)
    np.random.shuffle(x)
    np.random.seed(103)
    np.random.shuffle(x)

    # split data %80 train - %20 test
    x_test, x_train = split_data(x.copy())

    # create dictionary of unique words
    unique_words_dict, count_mails, total_words_dist = vectorizer(x_train.copy(), 1)

    # PART1
    max_prob_words_by_naive_bayes(unique_words_dict, total_words_dist)

    # PART2
    # calculate probabilities of all given test data
    results = calculate_probability(x_test.copy(), unique_words_dict, count_mails, total_words_dist, 1)

    # calculate performance of the given results
    accuracy, precision, recall, f1_score = calculate_performance(results)
    print("Unigram Accuracy: ", accuracy)
    print("Unigram Precision: ", precision)
    print("Unigram recall: ", recall)
    print("Unigram F1 score: ", f1_score)
    print()

    # BIGRAM
    # create dictionary of unique words
    unique_words_dict, count_mails, total_words_dist = vectorizer(x_train.copy(), 2)

    # PART1
    max_prob_words_by_naive_bayes(unique_words_dict, total_words_dist)

    # PART2
    # calculate probabilities of all given test data
    results = calculate_probability(x_test.copy(), unique_words_dict, count_mails, total_words_dist, 2)

    # calculate performance of the given results
    accuracy, precision, recall, f1_score = calculate_performance(results)
    print("Bigram Accuracy: ", accuracy)
    print("Bigram Precision: ", precision)
    print("Bigram recall: ", recall)
    print("Bigram F1 score: ", f1_score)
    print()

    # PART3
    print("TF-IDF")
    tf_idf(x_train.copy())

## **1. Part 1: Understanding the data**


For this part we implement a function named "max_prob_words_by_naive_bayes" (you can see in above code). In this function, we only consider frequencies of each word appeared in mails. Each words frequency calculated by count vectorizer whic is predefined in scikit learn library. By this vector we can calculate each words frequency in spam/ham mails. We calculate the frequencies for each word and make a dictionary with key value pair is shown as { word = [ spam count, ham count ] }. With those values we try to predict most usful keywords. When we try to figure out those keywords from our data, we try to select words which are not appeared often in both of the classes. If a word is appeared often in both classes, then this word has no effect on our prediction. 



With these values, we find 3 words which might be useful for us while predicting.
* Most frequent ham word : &emsp; **vince**  &emsp;&emsp;&emsp;(appeared 2248 times in ham mails)
* Second frequent ham word : &emsp;  **enron** &emsp;&emsp;(appeared 2081 times in ham mails)
* Third frequent ham word : &emsp; **cc**   &emsp;&emsp;&emsp;&emsp;(appeared 1765 times in ham mails)



* Most frequent spam word :  **here**  403
* Second frequent soam word :  **click** 246
* Third frequent spam word :  **000** 137

So, as we can see some words appeared more than the others. And this makes that specific word a better indicator of spam/ham mail. As we can see with our eyes too, those words appeared very often in their classes.For example the word "vince" appeared 2248 time in ham mails but there arent any appearence in spam mails. So that makes "vince" a very great indicator of being ham mail. By using those values, we can implement our naive bayes algorithm. As we can see from the words, naive bayes algorithm is appropriate and applicable. While some words indicate being ham, others could indicate being spam. With naive bayes algorithm we can calculate the probability of both cases and make comparison between them. We think naive bayes algorithm is applicable, by looking at the data.



## **2. Part 2: Implementing Naive Bayes**
You will represent your data with listed approaches and use them to learn a classifier via Naive Bayes algorithm. You have to implement your own Naive Bayes
algorithm.
• Features: You will use Bag of Words (BoW) model which learns a vocabulary from all of the documents, then models each document by counting the
number of times each word appears. You will use BoW with two options:
– Unigram: The occurrences of words in a document(frequency of the
word).
– Bigram: The occurrences of two adjacent words in a document.

## **3. Part 3:**

### **3.a) Analyzing effect of the words on prediction**
• List the 10 words whose presence most strongly predicts that the mail is
ham.


• List the 10 words whose absence most strongly predicts that the mail is
ham.


• List the 10 words whose presence most strongly predicts that the mail is
spam.


• List the 10 words whose absence most strongly predicts that the mail is
spam


### **3.b) Stopwords**
You may find common words like a, to, and others in your list in Part 3(a).
These are called stopwords. A list of stopwords is available in sklearn here.
You can import this as follows:
from sklearn.feature extraction.text import ENGLISH STOP WORDS
Now, list the 10 non-stopwords that most strongly predict that the mail is
ham, and the 10 non-stopwords that most strongly predict that the mail is
spam

### **3.c) Analyzing effect of the stopwords**
Why might it make sense to remove stop words when interpreting the model?
Why might it make sense to keep stop words?


## **4. Part 4: Calculation of Performance Metrics**
You will compute performance metrics below of your model to measure the success
of your classification method:

### **Unigram**

* Unigram Accuracy:  0.9921397379912664
* Unigram Precision:  0.9964994165694282
* Unigram recall:  0.9930232558139535
* Unigram F1 score:  0.9947582993593477

### **Bigram**

* Bigram Accuracy:  0.97117903930131
* Bigram Precision:  0.9975932611311673
* Bigram recall:  0.963953488372093
* Bigram F1 score:  0.9804849201655825

### **TF-IDF Vectorizer**

### **English Stop Words**